BDLE 2023

date du document  :  18/09/2023

# TP1 2023 Préparation de données  (ETUDIANT)


Préparation de données


## Préparation du TP

Vérifier que des ressources de calcul sont allouées à votre notebook  (cf RAM  de disque indiqués en haut à droite) . Sinon cliquer sur le bouton connecter pour obtenir des ressources.




Pour accéder directement aux fichiers stockées sur votre google drive. Renseigner le code d'authentification lorsqu'il est demandé

Ajuster le nom de votre dossier : MyDrive/ens/bdle/TP1

In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")

drive_dir = "/content/drive/MyDrive/ens/bdle/TP1"
os.makedirs(drive_dir, exist_ok=True)
os.listdir(drive_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[]

Installer pyspark et findspark (durée 1'):


In [ ]:
!pip install -q pyspark
!pip install -q findspark

Démarrer la session spark

In [ ]:
import os
import glob

pyspark_dir = glob.glob('/usr/local/lib/python*/dist-packages/pyspark')[0]
print("pyspark directory is", pyspark_dir)
os.environ["SPARK_HOME"] = pyspark_dir
os.environ["JAVA_HOME"] = "/usr"

pyspark directory is /usr/local/lib/python3.10/dist-packages/pyspark


In [ ]:
# Principaux import
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkConf

# pour les dataframe et udf
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G").\
  set("spark.sql.catalogImplementation","in-memory")

  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")

  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

session démarrée, son id est  local-1695812866972


In [ ]:
# on utilise 8 partitions au lieu de 200 par défaut
spark.conf.set("spark.sql.shuffle.partitions", "8")
print("Nombre de partitions utilisées : ", spark.conf.get("spark.sql.shuffle.partitions"))

Nombre de partitions utilisées :  8


In [ ]:
# Optionnel :
# pour l'accès à spark UI : voir https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip
# get_ipython().system_raw('./ngrok http 4050 &')
# !curl -s http://localhost:4040/api/tunnels

Redéfinir la fonction **display** pour afficher le resutltat des requêtes dans un tableau

In [ ]:
import pandas as pd
from google.colab import data_table

# alternatives to Databricks display function.

def display(df, n=100):
  return data_table.DataTable(df.limit(n).toPandas(), include_index=False, num_rows_per_page=10)

def display2(df, n=20):
  pd.set_option('max_columns', None)
  pd.set_option('max_colwidth', None)
  return df.limit(n).toPandas()


Définir le tag **%%sql** pour pouvoir écrire plus simplement des requêtes en SQL dans une cellule

In [ ]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

def removeComments(query):
  result = ""
  for line in query.split('\n'):
    if not(line.strip().startswith("--")):
      result += line + "\n"
  return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    derniere = None
    est_requete = False
    for r in tabRequetes:
        r = r.strip()
        if len(r) > 2:
          derniere = spark.sql(r)
          est_requete = ( r.lower().startswith('select')or r.lower().startswith('with'))
    if(est_requete):
      return display(derniere)
    else:
      return print('ok')

In [ ]:
# facultatif (à ne pas utiliser)
# %load_ext google.colab.data_table
# %unload_ext google.colab.data_table

## Accès aux données

### URL pour l'accès aux datasets

In [ ]:
# URL du dossier PUBLIC_DATASET contenant des fichiers de données pour les TP
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4"
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL du dossier contenant les datasets ", PUBLIC_DATASET_URL)

URL du dossier contenant les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


### Données de mobilité

Données issues du dataset YFCC

In [ ]:
local_dir = "/local/data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

['POI-Toro.csv', 'userVisits-Toro.csv', 'allCountries.zip', 'allCountries.txt']

In [ ]:
from urllib import request

# download dataset if not already donwloaded
def download_file(web_dir, local_dir, file):
  local_file = local_dir + "/" + file
  web_file = web_dir + "/" + file
  if(os.path.isfile(local_file)):
    print(file, "is already stored")
  else:
    print("downloading from URL: ", web_file , "save in : " + local_file)
    request.urlretrieve(web_file , local_file)

# user visits
web_dir = PUBLIC_DATASET + "YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/userVisits-ijcai15"
download_file(web_dir, local_dir, "userVisits-Toro.csv")

# poi
web_dir = PUBLIC_DATASET + "YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/poiList-ijcai15"
download_file(web_dir, local_dir, "POI-Toro.csv")



os.listdir(local_dir)

userVisits-Toro.csv is already stored
POI-Toro.csv is already stored


['POI-Toro.csv', 'userVisits-Toro.csv', 'allCountries.zip', 'allCountries.txt']

### Les visites

Lire les 2 premières lignes du  fichier csv en python.
Est ce que le fichier a une ligne d'entête ?
Quel caractère délimite deux valeurs consécutives dans une ligne de données ?

In [ ]:
f = open(local_dir + "/" + "userVisits-Toro.csv", "r")
print(f.readline()); print(f.readline())

"photoID";"userID";"dateTaken";"poiID";"poiTheme";"poiFreq";"seqID"

7941504100;"10007579@N00";1346844688;30;"Structure";1538;1



Lire le fichier des visites *sans* préciser le type des attributs.
Par défaut, tous les attributs sont considérés comme étant de type string.

In [ ]:
user_visits = spark.read.option("header", "True").option("delimiter", ";").format("csv").load(local_dir + "/" + "userVisits-Toro.csv")
user_visits.show(3)
user_visits.printSchema()

+----------+------------+----------+-----+---------+-------+-----+
|   photoID|      userID| dateTaken|poiID| poiTheme|poiFreq|seqID|
+----------+------------+----------+-----+---------+-------+-----+
|7941504100|10007579@N00|1346844688|   30|Structure|   1538|    1|
|4886005532|10012675@N05|1142731848|    6| Cultural|    986|    2|
|4886006468|10012675@N05|1142732248|    6| Cultural|    986|    2|
+----------+------------+----------+-----+---------+-------+-----+
only showing top 3 rows

root
 |-- photoID: string (nullable = true)
 |-- userID: string (nullable = true)
 |-- dateTaken: string (nullable = true)
 |-- poiID: string (nullable = true)
 |-- poiTheme: string (nullable = true)
 |-- poiFreq: string (nullable = true)
 |-- seqID: string (nullable = true)



Lire le fichier en précisant le schéma : nom et type des attributs

In [ ]:
schema = "photoID long, userID String, date Long, poiID int, poiTheme String, poiFreq int, seqID int"

user_visits = spark.read.option("header", "True").option("delimiter", ";").csv(local_dir + "/" + "userVisits-Toro.csv", schema = schema)
user_visits.persist()
user_visits.createOrReplaceTempView("user_visits")
user_visits.show(10)
user_visits.printSchema()

+----------+------------+----------+-----+---------+-------+-----+
|   photoID|      userID|      date|poiID| poiTheme|poiFreq|seqID|
+----------+------------+----------+-----+---------+-------+-----+
|7941504100|10007579@N00|1346844688|   30|Structure|   1538|    1|
|4886005532|10012675@N05|1142731848|    6| Cultural|    986|    2|
|4886006468|10012675@N05|1142732248|    6| Cultural|    986|    2|
|4885404441|10012675@N05|1142732373|    6| Cultural|    986|    2|
|4886008334|10012675@N05|1142732445|    6| Cultural|    986|    2|
|4886009150|10012675@N05|1142916492|    6| Cultural|    986|    3|
|7054481539|10012675@N05|1319327174|   13| Cultural|    964|    4|
|6908387594|10012675@N05|1319328255|   13| Cultural|    964|    4|
|6908381912|10012675@N05|1319331463|   13| Cultural|    964|    4|
|6908398496|10012675@N05|1319331886|   13| Cultural|    964|    4|
+----------+------------+----------+-----+---------+-------+-----+
only showing top 10 rows

root
 |-- photoID: long (nullable = 

Vérifier qu'il n'y a pas de date nulle

In [ ]:
%%sql
select count(1) as nb_dates_nulles
from user_visits
where date is null

-- autre solution avec "case"
--select count(case when isnull(date) then date end) as nb_date_nulles
--from user_visits

,nb_dates_nulles
0,0



Vérifier qu'il n'y a aucune séquence associée à plusieurs utilisateurs

In [ ]:
%%sql
select seqID
from user_visits
group by seqID
having count(distinct userID) > 1

,seqID



#### POI_sequence

La table **POI_sequence**(seqID, poiID). Les POI visités durant une séquence.
Pour simplifier on compte chaque POI une seule fois par séquence et on ne considère pas l'ordre de visite des POI.

Rmq: il n'est pas nécessaire de préciser le userID pour identifier une séquence.

In [ ]:
%%sql
create or replace temp view POI_sequence as
select distinct seqID, poiID
from user_visits;

select * from POI_sequence

,seqID,poiID
0,9,24
1,16,24
2,26,22
3,29,25
4,54,25
...,...,...
95,624,23
96,634,22
97,638,4
98,641,30


### Les lieux visités : POI
Ils sont appelés *Point Of Interest*

In [ ]:
poi_schema = "poiID long, poiName String, latitude double, longitude double, theme String"

poi = spark.read.option("header", "True").option("delimiter", ";").csv(local_dir + "/" + "POI-Toro.csv", schema = poi_schema)
poi.show(3)
poi.printSchema()
poi.createOrReplaceTempView("POI")

+-----+------------------+--------+---------+-----+
|poiID|           poiName|latitude|longitude|theme|
+-----+------------------+--------+---------+-----+
|    1| Air_Canada_Centre|43.64333|-79.37917|Sport|
|    2|         BMO_Field|43.63278|-79.41861|Sport|
|    3|Maple_Leaf_Gardens|43.66222|-79.38028|Sport|
+-----+------------------+--------+---------+-----+
only showing top 3 rows

root
 |-- poiID: long (nullable = true)
 |-- poiName: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- theme: string (nullable = true)



In [ ]:
%%sql
cache table POI;

SELECT *
FROM POI

,poiID,poiName,latitude,longitude,theme
0,1,Air_Canada_Centre,43.643330,-79.379170,Sport
1,2,BMO_Field,43.632780,-79.418610,Sport
2,3,Maple_Leaf_Gardens,43.662220,-79.380280,Sport
3,4,Rogers_Centre,43.641390,-79.389170,Sport
4,5,Woodbine_Racetrack,43.712525,-79.602042,Sport
5,6,Art_Gallery_of_Ontario,43.653890,-79.392780,Cultural
6,7,Hockey_Hall_of_Fame,43.646976,-79.377253,Cultural
7,8,Ripley%27s_Aquarium_of_Canada,43.642481,-79.386050,Cultural
8,9,Ontario_Science_Centre,43.716670,-79.338330,Cultural
9,10,Riverdale_Farm,43.667111,-79.361294,Cultural


la liste des thèmes

In [ ]:
%%sql
create or replace temp view themes as
select distinct theme
from POI
order by theme;

cache table themes;

select *
from themes

,theme
0,Amusement
1,Beach
2,Cultural
3,Shopping
4,Sport
5,Structure


## Exercice 1

#### 1) Les 10 POI les plus photographiés
indications: on peut compléter une requête SQL avec *LIMIT 10* pour borner la taille résultat (partiel) souhaité.

In [ ]:
%%sql
SELECT poiID, count(*) as fq
FROM user_visits
GROUP BY poiID
Order by fq desc
limit 10


,poiID,fq
0,11,4139
1,22,3603
2,21,3591
3,16,3553
4,1,3506
5,4,3056
6,7,2053
7,23,1866
8,8,1736
9,25,1701


#### 2) Les visites avec date détaillée
Définir la table Visite_date(userID, seqID, poiID, date, annee, mois, jour, heure) contenant les visites avec la date détaillée composée des attributs : année, mois, jour, heure.
Indication : la date est initialement au format "unix". Voir la fonction de conversion de date from_unixtime et la fonction extract.

In [ ]:
%%sql
SELECT *
FROM user_visits
limit 10

,photoID,userID,date,poiID,poiTheme,poiFreq,seqID
0,7941504100,10007579@N00,1346844688,30,Structure,1538,1
1,4886005532,10012675@N05,1142731848,6,Cultural,986,2
2,4886006468,10012675@N05,1142732248,6,Cultural,986,2
3,4885404441,10012675@N05,1142732373,6,Cultural,986,2
4,4886008334,10012675@N05,1142732445,6,Cultural,986,2
5,4886009150,10012675@N05,1142916492,6,Cultural,986,3
6,7054481539,10012675@N05,1319327174,13,Cultural,964,4
7,6908387594,10012675@N05,1319328255,13,Cultural,964,4
8,6908381912,10012675@N05,1319331463,13,Cultural,964,4
9,6908398496,10012675@N05,1319331886,13,Cultural,964,4


In [ ]:
%%sql
SELECT *
FROM POI
limit 10

,poiID,poiName,latitude,longitude,theme
0,1,Air_Canada_Centre,43.643330,-79.379170,Sport
1,2,BMO_Field,43.632780,-79.418610,Sport
2,3,Maple_Leaf_Gardens,43.662220,-79.380280,Sport
3,4,Rogers_Centre,43.641390,-79.389170,Sport
4,5,Woodbine_Racetrack,43.712525,-79.602042,Sport
5,6,Art_Gallery_of_Ontario,43.653890,-79.392780,Cultural
6,7,Hockey_Hall_of_Fame,43.646976,-79.377253,Cultural
7,8,Ripley%27s_Aquarium_of_Canada,43.642481,-79.386050,Cultural
8,9,Ontario_Science_Centre,43.716670,-79.338330,Cultural
9,10,Riverdale_Farm,43.667111,-79.361294,Cultural


In [ ]:
%%sql
create or replace temp view Visite_Date as
select userID, seqID, poiID, from_unixtime(date) as date, year(from_unixtime(date)) as annee, month(from_unixtime(date)) as mois, day(from_unixtime(date)) as jour, hour(from_unixtime(date)) as heure, minute(from_unixtime(date)) as minutes
from user_visits
;

select * from Visite_Date

,userID,seqID,poiID,date,annee,mois,jour,heure,minutes
0,10007579@N00,1,30,2012-09-05 11:31:28,2012,9,5,11,31
1,10012675@N05,2,6,2006-03-19 01:30:48,2006,3,19,1,30
2,10012675@N05,2,6,2006-03-19 01:37:28,2006,3,19,1,37
3,10012675@N05,2,6,2006-03-19 01:39:33,2006,3,19,1,39
4,10012675@N05,2,6,2006-03-19 01:40:45,2006,3,19,1,40
...,...,...,...,...,...,...,...,...,...
95,10014440@N06,10,25,2008-07-09 08:53:33,2008,7,9,8,53
96,10014440@N06,10,25,2008-07-09 08:53:54,2008,7,9,8,53
97,10014440@N06,10,25,2008-07-09 08:54:10,2008,7,9,8,54
98,10014440@N06,10,25,2008-07-09 08:54:15,2008,7,9,8,54


#### 3a) Le nombre de POI par utilisateur
Le résultat (userID, nbPOI) est trié par nombre décroissant de POI.

In [ ]:
%%sql
SELECT userID, count(poiID) as nbpoi
FROM user_visits
GROUP BY userID
ORDER BY nbpoi desc


,userID,nbpoi
0,48784629@N00,3265
1,39460517@N03,1730
2,7616875@N04,1665
3,84987970@N00,1430
4,86822950@N03,1403
...,...,...
95,91008793@N00,68
96,70307237@N00,67
97,13833167@N00,66
98,27914970@N04,65


#### 3b) Le nombre de POI par séquence
Définir la table Seq3plus(seqID, nbPoi) correspondant aux séquences qui contiennent au moins 3 POI distincts.
Afficher le résultat trié par nombre de POI décroissant puis numéro de séquence croissant.


In [ ]:
%%sql
create or replace temp view Seq3plus as
SELECT SeqID, count(DISTINCT poiID) as nbpoi
FROM user_visits
GROUP BY seqID
having nbpoi >= 3;


select * from seq3plus
order by nbPOI desc, seqID;

,SeqID,nbpoi
0,298,13
1,4961,10
2,4351,9
3,5964,9
4,510,8
...,...,...
95,2377,4
96,2466,4
97,2472,4
98,2896,4


#### 4) Trajectoire



a) Définir une fonction qui prend en entrée une liste de couples (date, poi) et qui retourne une liste PoI triés par date croissante. Le resultat ne contient pas 2 PoI consécutifs identiques.

In [ ]:
def trierPOI( L):
  result = sorted (L, key = lambda x: x[0])

  return [result[i][1] for i in range(len(result)) if i== 0 or result[i][1] != result[i-1][1]]

spark.udf.register("trierPOI", trierPOI, ArrayType(IntegerType()))

##test local
print(trierPOI([(13,0),(10,2), (12,1), (14,2),(11,2)]))

[2, 1, 0, 2]


Définir la vue Trajectoire(userID, seqID, listPOI).

listPOI est la liste des POI visités, pendant la séquence seqID, dans l'ordre chronologique

Indications:
pour la sequence 687 la listPOI est [7,16,4,8,4,16]

In [ ]:
%%sql
create or replace temp view Trajectoire as
SELECT userID, seqID, trierPOI(collect_list((date, poiID))) as listePOI
FROM user_visits
GROUP BY userID, seqID

;


cache table Trajectoire;

select * from Trajectoire
where size(listePOI) =6
order by seqID

,userID,seqID,listePOI
0,14617133@N05,384,"[22, 28, 22, 28, 22, 28]"
1,20456447@N03,687,"[7, 16, 4, 8, 4, 16]"
2,20741443@N00,729,"[23, 21, 22, 7, 28, 23]"
3,27168489@N00,1358,"[16, 8, 28, 22, 23, 21]"
4,29352917@N00,1524,"[29, 30, 16, 29, 16, 29]"
5,30624156@N00,1598,"[28, 7, 30, 28, 22, 27]"
6,32827327@N03,1786,"[20, 6, 25, 11, 30, 24]"
7,33547369@N00,1877,"[23, 21, 23, 22, 23, 22]"
8,34128007@N04,1912,"[8, 16, 4, 1, 29, 16]"
9,34314322@N00,1993,"[30, 22, 28, 29, 30, 16]"


#### 5) Transitions
5a) Définir la table Transitions(poi1, poi2, nbTrans) avec nbTrans étant le nombre de déplacements directs de poi1 à poi2.
Indication: deux POI apparaissant successivement (poi1 suivi de poi2) dans une séquence forment un déplacement direct entre poi1 et poi2.
Une séquence contenant un seul POI ne correspond à aucun déplacement.

In [ ]:
from collections import Counter

def transition(listePOI):
  L=[]
  for i in range(1,len(listePOI)):
    L.append((listePOI[i-1], listePOI[i]))
  return L



spark.udf.register("transition", transition, ArrayType(StructType([StructField("poi1", IntegerType() ), StructField("poi2", IntegerType())] )))
#, StructField("nb", int)

<function __main__.transition(listePOI)>

In [ ]:
%%sql
create or replace temp view Transition1 as
SELECT poi1poi2.poi1 as poi1, poi1poi2.poi2 as poi2, count(*) as nbTrans
FROM (SELECT explode(transition(listePOI)) as poi1poi2
FROM Trajectoire
WHERE size(listePOI) > 1 )
GROUP BY poi1, poi2
;

cache table Transition1;

select *
from Transition1
order by poi1

,poi1,poi2,nbTrans
0,1,7,5
1,1,24,2
2,1,4,1
3,1,22,3
4,1,30,14
...,...,...,...
95,11,30,2
96,13,16,4
97,13,21,2
98,13,4,1


b) Definir la vue
Transition_relative avec des valeurs de transition relatives au nombre total de transition partant d'un POI.

Indication: créer une vue donnant le nombre total de transitions par poi.

In [ ]:
%%sql
create or replace temp view Total_transition as
SELECT poi1, sum(nbTrans) as nbTransTot
FROM Transition1
GROUP BY poi1

;

cache table Total_transition ;

select *
from Total_transition
order by poi1

,poi1,nbTransTot
0,1,45
1,2,18
2,3,24
3,4,61
4,6,61
5,7,153
6,8,93
7,9,1
8,10,4
9,11,68


In [ ]:
%%sql
create or replace temp view Transition_relative as
SELECT T1.poi1, T1.poi2, T1.nbTrans , T1.nbTrans/T2.nbTransTot as nbTransRel
FROM Transition1 T1, Total_transition T2
WHERE T1.poi1 = T2.poi1
;

select *
from Transition_relative
ORDER BY poi1

,poi1,poi2,nbTrans,nbTransRel
0,1,7,5,0.111111
1,1,24,2,0.044444
2,1,4,1,0.022222
3,1,22,3,0.066667
4,1,30,14,0.311111
...,...,...,...,...
95,11,30,2,0.029412
96,13,16,4,0.200000
97,13,21,2,0.100000
98,13,4,1,0.050000


### 6) DuréeVisitePOI
La durée moyenne de visite d'un POI.
Indication, pour une série d'événements consécutifs associés à un même POI dans une séquence, la durée de visite est la différence de date entre le 1er et le dernier événement.

In [ ]:
def dureeVisite(liste_couples):
  result =[]
  d = 1
  for i in range(1,len(liste_couples)):
    if liste_couples[i][0] == liste_couples[i-1][0] and d==1:
      debut = liste_couples[i-1][1]
      d+=1
    if d != 1 and liste_couples[i][0] != liste_couples[i-1][0]  :
      result.append((liste_couples[i-1][0], liste_couples[i-1][1]-debut))
      début = 0
      d=1
    if d != 1 and liste_couples[i] == liste_couples[-1]:
      result.append((liste_couples[i][0], liste_couples[i][1]-debut))
  return result

spark.udf.register("dureeVisite", dureeVisite, ArrayType(StructType([StructField("poi", IntegerType()),StructField("duree", IntegerType())] )))

# # test local
print(dureeVisite([(102,8), (102,11), (106,14) ,(101,15), (101,17), (102,20), (102,21), (102,24)]))

[(102, 3), (101, 2), (102, 4)]


In [ ]:
%%sql
create or replace temp view DureeVisitePOI as
SELECT userID, seqID, nupl.poi as poi, nupl.duree as duree
FROM (
SELECT userID, seqID, explode(listeDuree) as nupl
FROM (
SELECT  userID, seqID, dureeVisite(collect_list((poiID,date))) as listeDuree
FROM user_visits
GROUP BY userID, seqID
having size(listeDuree) > 1
)
)
;


select *
from DureeVisitePOI d
order by userID, seqID

,userID,seqID,poi,duree
0,10014440@N06,8,23,1615
1,10014440@N06,8,24,1359
2,10218635@N06,33,21,0
3,10218635@N06,33,25,4049
4,10502709@N05,58,7,5302
...,...,...,...,...
95,16125685@N04,512,7,1153
96,16125685@N04,512,21,1427
97,16125685@N04,513,23,148
98,16125685@N04,513,21,6876


In [ ]:
%%sql
create or replace temp view DureeVisitePOI as
SELECT userID, seqID, nupl.poi as poi, nupl.duree as duree
FROM (
SELECT userID, seqID, explode(listeDuree) as nupl
FROM (
SELECT userID, seqID, dureeVisite(collect_list((poiID,jour*60*24+heure*60+minutes))) as listeDuree
FROM Visite_Date
GROUP BY userID, seqID
having size(listeDuree) >=1
order by userID, seqID
)
)
;

select *
from DureeVisitePOI d
order by userID, seqID

,userID,seqID,poi,duree
0,10012675@N05,2,6,10
1,10012675@N05,4,13,94
2,10014440@N06,5,24,4
3,10014440@N06,6,23,18
4,10014440@N06,7,23,9
...,...,...,...,...
95,11090433@N05,139,3,5
96,11090433@N05,139,23,2
97,11090433@N05,139,22,5
98,11090433@N05,139,30,361


In [ ]:
%%sql
create or replace temp view DureeMoyenneVisitePOI as
SELECT poi, mean(duree) as dureeMoyenne
FROM DureeVisitePOI
GROUP BY poi
;

select *
from DureeMoyenneVisitePOI

,poi,dureeMoyenne
0,13,55.029412
1,12,82.163636
2,14,26.666667
3,18,51.800000
4,6,50.192982
5,23,50.305785
6,17,159.950000
7,16,69.820331
8,9,69.542857
9,10,61.500000


### 7) Durée moyenne de déplacement entre deux POIs.

Définir la vue **DureeDeplacement** (poi1, poi2, duree) calculant la duree moyenne de déplacement entre deux POI consécutifs dans une séquence.



In [ ]:
%%sql

create or replace temp view DureeMoyenneDeplacement as
SELECT a.poiID AS poi1, b.poiID AS poi2, AVG(b.date - a.date)/60 AS duree_en_min
FROM user_visits a
JOIN user_visits b
ON a.userID = b.userID AND a.seqID = b.seqID AND a.date < b.date
GROUP BY a.poiID, b.poiID
;


select *
from DureeMoyenneDeplacement





,poi1,poi2,duree_en_min
0,23,24,89.857658
1,11,27,67.623785
2,27,27,22.089880
3,28,23,89.142249
4,25,6,101.218605
...,...,...,...
95,17,17,188.808033
96,7,7,27.340415
97,11,11,54.285990
98,22,22,157.217335


## Exercice 2

Charger les données de Geonames

In [ ]:
import zipfile

#geonames
web_dir = PUBLIC_DATASET + "geonames_ALL"

download_file(web_dir, local_dir, "allCountries.zip")

#unzip
local_file = local_dir + "/" + "allCountries.txt"
if(os.path.isfile(local_file)):
    print(local_file, "is already stored")
else:
  with zipfile.ZipFile(local_dir + "/" + "allCountries.zip", 'r') as zip_ref:
    zip_ref.extractall(local_dir)

os.listdir(local_dir)

In [ ]:
geonames =  spark.read.option("header", "False").option("delimiter", "\t").format("csv").load(local_dir + "/" + "allCountries.txt")
geonames.createOrReplaceTempView("geonames")
geonames.show(3)

### 1) Geonames
Définir le schéma, limité au 9 premiers attributs donnant des informations sur l'identifiant d'un POI, ses noms, sa position GPS, le code du pays..., ), d'après le document [readme.txt](https://nuage.lip6.fr/s/kaBX3sF3YEzMZHK) dans le dossier [geonames_ALL](https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4?path=%2Fgeonames_ALL)

#####1a) schéma de Geonames

Définir Geonames2 en précisant le schéma : nom et type des attributs.
Se limiter au 9 premiers attributs donnant des informations sur l'identifiant d'un POI, ses noms, sa position GPS, le code du pays..., ), d'après le document [readme.txt](https://nuage.lip6.fr/s/kaBX3sF3YEzMZHK) dans le dossier [geonames_ALL](https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4?path=%2Fgeonames_ALL)

In [ ]:
%%sql
create or replace temp view Geonames2 as
select cast(_c0 as long) as geonameID,
cast(_c1 as string) as name,
cast(_c2 as string) as asciiname,
cast(_c3 as string) as alternatenames,
cast(_c4 as double) as latitude,
cast(_c5 as double) as longitude,
cast(_c6 as string) as feature_class,
cast(_c7 as string) as feature_code,
cast(_c8 as string) as country_code


from Geonames;

select * from Geonames2

##### 1b) Extrait pour le Canada

Définir la vue Geonames_canada pour les POI situés au Canada.

In [ ]:
%%sql
create or replace temp view Geonames_canada as
SELECT *
FROM Geonames2
WHERE country_code like "CA"

;

cache table Geonames_canada;

select * from  Geonames_canada;

### 2) Association entre les POI et Geonames
Compléter les POI avec des attributs de geonames (par exemple name et [feature code](http://www.geonames.org/export/codes.html))

Décrire la solution que vous proposez.

Imaginons que nous souhaitions obtenir le feature code pour un poi de la première BDD, il va falloir faire une jointure.
Dans la première base de données, le poiID ne correspond pas avec le geonameID donc il est impossible de faire une jointure la dessus.

On pourrait faire une jointure en se basant sur la latitude et la longitude qui sont présente sur les deux BDD. Normalement pour un POI celles ci sont uniques. Bien sur sous condition que celles ci soit toutes arrondies à la meme decimale.
On voit que ce n'est pas parfait car si on met 4 decimal on a plus qu'un seul tuple qui sort, si on met 3 certains POI on plusieurs feature code .

On pourrait imaginer faire du REGEX sur le nom pour pouvoir faire une jointure dessus.



In [ ]:
%%sql
create or replace temp view Geonames_canada2 as
SELECT ROUND(latitude,3) as latitude, ROUND(longitude,3) as longitude, feature_code FROM Geonames_canada
;
cache table Geonames_canada2;
create or replace temp view POI2 as
SELECT ROUND(latitude,3) as latitude, ROUND(longitude,3) as longitude, poiName FROM POI
;
cache table POI2;

In [ ]:
%%sql
SELECT poiName, feature_code
FROM Geonames_canada2 G , POI2 P
WHERE G.latitude = P.latitude and G.longitude =P.longitude